# Portfolio Optimisation bot

markowitz portfolio theory: create a portfolio which maximises the return for a given amount of risk.
either invest in traditional safe havens or use diversification.

expected return: $E(R_p) = \sum_{i} w_i E(R_i)$ where $w_i$ is the weight for the $i^{th}$ asset and $R_i$ is the return on the $i^{th}$ asset. 

portfolio return: $\sigma^2_p = \sum_i w^2_i \sigma^2_i + \sigma_i \sigma_{j \neq i} w_i w_j \sigma_i \sigma_j \rho_{ij}$ where $\sigma_i$ is the riskiness of the $i^{th}$ asset and $\rho_{ij}$ is the correlation coefficient between asset $i$ and asset $j$.

sharpe ratio: $\frac{R_p - R_f}{\sigma_p}$, where $R_p = $ return of portfolio, $R_f =$ risk-free rate, $\sigma_p = $standard deviation of portfolio's excess return.
In general, a higher sharpe ratio indicates a better and more lucrative investment.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as web #lookup
from matplotlib.ticker import FuncFormatter #lookup more
import pypfopt


In [3]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
#from pypfopt.plotting import Plotting

# scrapping data from web

pull data from yahoo! finance


In [4]:
#tickers = ['BSX', 'AES','BRK','SEE','QQQ','SPY']
tickers = ['KR', 'PLD','DTE','CME','SONY','BABA']
thelen = len(tickers)
price_data = []

for ticker in range(thelen):
    prices = web.DataReader(tickers[ticker], start = '2018-01-01',end='2021-07-07', data_source='yahoo')
    price_data.append(prices.assign(ticker=ticker)[['Adj Close']])
df_stocks = pd.concat(price_data, axis=1)
df_stocks.columns=tickers
df_stocks.head()

,KR,PLD,DTE,CME,SONY,BABA
Date,,,,,,
2018-01-02,26.296549,58.322220,81.927666,131.867477,45.790001,183.649994
2018-01-03,25.878407,58.212894,81.663269,133.980453,46.299999,184.000000
2018-01-04,25.069998,57.046631,80.303474,135.410294,47.250000,185.710007
2018-01-05,25.385927,57.001072,80.227921,136.293716,48.700001,190.699997
2018-01-08,25.989908,57.666203,80.817154,138.188080,49.000000,190.330002


check for any NaN values in the data

In [5]:
nullin_df = pd.DataFrame(df_stocks,columns=tickers)
print(nullin_df.isnull().sum())

KR      0
PLD     0
DTE     0
CME     0
SONY    0
BABA    0
dtype: int64


In [6]:
df_stocks

,KR,PLD,DTE,CME,SONY,BABA
Date,,,,,,
2018-01-02,26.296549,58.322220,81.927666,131.867477,45.790001,183.649994
2018-01-03,25.878407,58.212894,81.663269,133.980453,46.299999,184.000000
2018-01-04,25.069998,57.046631,80.303474,135.410294,47.250000,185.710007
2018-01-05,25.385927,57.001072,80.227921,136.293716,48.700001,190.699997
2018-01-08,25.989908,57.666203,80.817154,138.188080,49.000000,190.330002
...,...,...,...,...,...,...
2021-06-30,38.310001,119.529999,110.297874,212.679993,97.220001,226.779999
2021-07-01,38.130001,119.980003,111.879997,214.199997,98.250000,221.869995
2021-07-02,38.369999,121.029999,111.739998,213.520004,101.650002,217.750000


In [7]:
df_stocks = df_stocks.dropna()
df_stocks

,KR,PLD,DTE,CME,SONY,BABA
Date,,,,,,
2018-01-02,26.296549,58.322220,81.927666,131.867477,45.790001,183.649994
2018-01-03,25.878407,58.212894,81.663269,133.980453,46.299999,184.000000
2018-01-04,25.069998,57.046631,80.303474,135.410294,47.250000,185.710007
2018-01-05,25.385927,57.001072,80.227921,136.293716,48.700001,190.699997
2018-01-08,25.989908,57.666203,80.817154,138.188080,49.000000,190.330002
...,...,...,...,...,...,...
2021-06-30,38.310001,119.529999,110.297874,212.679993,97.220001,226.779999
2021-07-01,38.130001,119.980003,111.879997,214.199997,98.250000,221.869995
2021-07-02,38.369999,121.029999,111.739998,213.520004,101.650002,217.750000


In [8]:
nullin_df = pd.DataFrame(df_stocks,columns=tickers)
print(nullin_df.isnull().sum())

KR      0
PLD     0
DTE     0
CME     0
SONY    0
BABA    0
dtype: int64


# calculations: return, variance and sharpe ratio

In [9]:
mu = expected_returns.mean_historical_return(df_stocks)
sigma = risk_models.sample_cov(df_stocks)
ef = EfficientFrontier(mu, sigma, weight_bounds = (-1,1))
sharpe_pfolio = ef.max_sharpe() #gives weights
sharpe_pwt = ef.clean_weights()
ef.portfolio_performance(verbose=True)

print(sharpe_pwt)

Expected annual return: 37.1%
Annual volatility: 34.7%
Sharpe Ratio: 1.01
OrderedDict([('KR', 0.12917), ('PLD', 0.84885), ('DTE', -0.45884), ('CME', 0.11635), ('SONY', 0.78947), ('BABA', -0.425)])
